In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('school_dataset_sample.csv')

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (306) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df = df[df['year']==2020]

In [4]:
df = df.reset_index()

In [5]:
col = [
       'sexM', 'sexF', 'phys_mat', 'foreign_phyl',
       'universal', 'ukr_phyl', 'technology', 'bio_phys', 'economy', 'history',
       'sport', 'math', 'law', 'other', 'martial_sport', 'philosophy',
       'ecology', 'arts', 'geography', 'physics', 'chem_tech', 'biology',
       'phys_chem', 'young_spec', 'qual_work', 'hung_lang_max', 'ukr_lang_max',
       'rus_lang_max', 'roman_lang_max', 'pol_lang_max', 'mold_lang_max',
       'other_lang_max', 'eotypename_min', 'eoregname_min', 'eoareaname_min',
       'eotername_min', 'eoparent_min',
       'mathball100_mean', 
       'Ступінь закладу ( I, I-II, I-III)',
       'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)',
       'Кількість штатних працівників педагогічного персоналу  разом',
       'Кількість штатних працівників непедагогічного персоналу  разом',
       'Кількість штатних працівників педагогічного персоналу  пенсійного віку',
       'Питома вага непедагогічного персоналу у загальній кількості працівників, %**',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**',
       'Кількість учнів разом', 'Кількість класів разом',
       'Фактична наповнюваність класів, учнів на клас**',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**',
       'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)',
       'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)',
       'Видатки на 1 учня, 2018, тис. грн.**',
       'Видатки на 1 клас, тис. грн.** 2018 рік (каса)',
       'Тип школи: міська / сільська',
       'Разом кількість учнів, та дітей, які здобувають дошкільну освіту',
       'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)',
       'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)',
       'Видатки на 1 учня, 2019, тис. грн.**',
       'Видатки на 1 клас, тис. грн.** 2019 рік (каса)',

       'Питома вага непедагогічного персоналу у загальній кількості працівників, %',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %',
       'Фактична наповнюваність класів, учнів на клас',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя',
]

In [9]:
pd.set_option('display.max_columns', None)
df[col].head()

,sexM,sexF,phys_mat,foreign_phyl,universal,ukr_phyl,technology,bio_phys,economy,history,sport,math,law,other,martial_sport,philosophy,ecology,arts,geography,physics,chem_tech,biology,phys_chem,young_spec,qual_work,hung_lang_max,ukr_lang_max,rus_lang_max,roman_lang_max,pol_lang_max,mold_lang_max,other_lang_max,eotypename_min,eoregname_min,eoareaname_min,eotername_min,eoparent_min,mathball_mean,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,Кількість штатних працівників педагогічного персоналу пенсійного віку,"Питома вага непедагогічного персоналу у загальній кількості працівників, %**","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**",Кількість учнів разом,Кількість класів разом,"Фактична наповнюваність класів, учнів на клас**","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)","Видатки на 1 учня, 2018, тис. грн.**","Видатки на 1 клас, тис. грн.** 2018 рік (каса)",Тип школи: міська / сільська,"Разом кількість учнів, та дітей, які здобувають дошкільну освіту","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)","Питома вага непедагогічного персоналу у загальній кількості працівників, %","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %","Фактична наповнюваність класів, учнів на клас","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя"
0,6,3,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ліцей,Луганська область,Троїцький район,с.Демино-Олександрівка,"Відділ освіти, молоді та спорту Троїцької сели...",12.857143,I-III,NaN,15.0,6.0,2.0,NaN,NaN,62,8,NaN,NaN,NaN,NaN,NaN,NaN,село,NaN,NaN,NaN,NaN,NaN,0.285714,0.133333,7.750000,4.133333
1,2,2,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ліцей,Луганська область,Троїцький район,с.Лантратівка,"Відділ освіти, молоді та спорту Троїцької сели...",19.666667,I-III,NaN,17.0,9.0,4.0,NaN,NaN,96,11,NaN,NaN,NaN,NaN,NaN,NaN,село,NaN,NaN,NaN,NaN,NaN,0.346154,0.235294,8.727273,5.647059
2,23,25,0,19,0,0,0,0,0,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,навчально-виховний комплекс,м.Київ,м.Київ. Шевченківський район міста,Шевченківський район міста,Управління освіти Шевченківської районної в мі...,40.625000,I-III,NaN,53.0,34.0,14.0,NaN,NaN,782,27,NaN,NaN,NaN,NaN,NaN,NaN,місто,NaN,NaN,NaN,NaN,NaN,0.390805,0.264151,28.962963,14.754717
3,11,10,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ліцей,Луганська область,Троїцький район,смт Троїцьке,"Відділ освіти, молоді та спорту Троїцької сели...",14.642857,I-III,NaN,41.0,25.0,5.0,NaN,NaN,362,21,NaN,NaN,NaN,NaN,NaN,NaN,місто,NaN,NaN,NaN,NaN,NaN,0.378788,0.121951,17.238095,8.829268
4,5,4,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,середня загальноосвітня школа,Сумська область,Глухівський район,смт Шалигине,"Відділ освіти, молоді та спорту Шалигинської с...",9.250000,I-III,NaN,29.0,22.0,4.0,NaN,NaN,149,15,NaN,NaN,NaN,NaN,NaN,NaN,місто,NaN,NaN,NaN,NaN,NaN,0.431373,0.137931,9.933333,5.137931


In [19]:
from asyncio import constants
import numpy as np
import pandas as pd
from typing import Tuple
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import *


def joint_sort_descending(l1:np.array, 
                          l2:np.array)->Tuple[np.array]:
    # l1 and l2 have to be numpy arrays
    idx = np.argsort(l1)[::-1]
    return l1[idx], l2[idx]

def generate_binary_at_k(y_scores:list, 
                         k:float) -> list:
    cutoff_index = int(len(y_scores) * (k / 100.0))
    test_predictions_binary = [1 if x < cutoff_index else 0 for x in range(len(y_scores))]
    return test_predictions_binary

def metrics_at_k(y_true: list, 
                   y_scores: list, 
                   k: float) -> float:
    y_scores, y_true = joint_sort_descending(np.array(y_scores), np.array(y_true))
    preds_at_k = generate_binary_at_k(y_scores, k)
    precision = precision_score(y_true, preds_at_k)
    recall = recall_score(y_true, preds_at_k)
    return precision,recall


class Preprocessing:
    DATA_TYPES = ['train','test']

    def __init__(self,
                 drop_strange_st_data: bool = True,
                 column_for_onehot: list = [],
                 scale_columns: list = []):

        self.drop_strange_st_data = drop_strange_st_data
        self.column_for_onehot = column_for_onehot
        self.scale_columns = scale_columns

        self.encoder = OneHotEncoder()
        self.scaler = MinMaxScaler()
        self.is_encoder_fit = False
        self.is_scaler_fit = False

    def process(self, 
                df: pd.DataFrame,
                data_type: str = 'train') -> pd.DataFrame:

        if data_type not in self.DATA_TYPES:
            raise ValueError(f'Unexpected data type. Data type must be one of {self.DATA_TYPES}.')
        
        if ((not self.is_encoder_fit) and (self.column_for_onehot)) and data_type == 'test':
            raise ValueError(f'Please, use train dataset first.')
            
        if ((not self.is_scaler_fit) and (self.scale_columns)) and data_type == 'test':
            raise ValueError(f'Please, use train dataset first.')

        if self.drop_strange_st_data:
            df = self._drop_strange_data_students_data(df)
        
        if self.column_for_onehot:
            df = self._one_hot_encoding(
                df = df,
                column_for_onehot = self.column_for_onehot,
                data_type = data_type
            )
            self.is_encoder_fit = True
        
        if self.scale_columns:
            df = self._scaling_columns(
                df = df,
                scale_columns = self.scale_columns,
                data_type = data_type
            )
            self.is_scaler_fit = True

        object_columns = self._get_object_columns(df)
        df,obj2num = self._encode_str_values(
            df = df,
            obj_columns = object_columns
        )

        self.obj2num = obj2num

        return df

    def _one_hot_encoding(self,
                          df: pd.DataFrame,
                          column_for_onehot: list,
                          data_type: str) -> pd.DataFrame:

        if data_type == 'train':
            data = self.encoder.fit_transform(df[column_for_onehot]).toarray()

        else: 
            data = self.encoder.transform(df[column_for_onehot]).toarray()

        df[self.encoder.get_feature_names_out(column_for_onehot)] = data
        df = df.drop(columns = column_for_onehot)

        return  df
    
    def _scaling_columns(self,
                         df: pd.DataFrame,
                         scale_columns: list,
                         data_type: str) -> pd.DataFrame:
        if data_type == 'train':
            df[scale_columns] = self.scaler.fit_transform(df[scale_columns])
        else: 
            df[scale_columns] = self.scaler.transform(df[scale_columns])

        return df
    
    @staticmethod
    def _encode_str_values(df: pd.DataFrame,
                           obj_columns: list) -> pd.DataFrame:
        obj2num = []
        for col in obj_columns:
            obj2num.append({col:{obj: i for i,obj in enumerate(set(df[col]))}})
            cur_num = obj2num[-1]
            df[col] = df[col].map(lambda x:cur_num[col][x])
        
        return (df,obj2num)

    @staticmethod
    def _drop_strange_data_students_data(df: pd.DataFrame) -> pd.DataFrame:
        return df.reset_index(drop=True)
    
    @staticmethod
    def _get_object_columns(df:pd.DataFrame) -> list:
        obj_col = []
        for column in df.columns:
            if df[column].dtype == 'object':
                obj_col.append(column)

        return obj_col

In [21]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Regression

In [22]:
X = df[col]

In [23]:
X = X.fillna(-1)

In [24]:
X.head()

,sexM,sexF,phys_mat,foreign_phyl,universal,ukr_phyl,technology,bio_phys,economy,history,sport,math,law,other,martial_sport,philosophy,ecology,arts,geography,physics,chem_tech,biology,phys_chem,young_spec,qual_work,hung_lang_max,ukr_lang_max,rus_lang_max,roman_lang_max,pol_lang_max,mold_lang_max,other_lang_max,eotypename_min,eoregname_min,eoareaname_min,eotername_min,eoparent_min,mathball_mean,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,Кількість штатних працівників педагогічного персоналу пенсійного віку,"Питома вага непедагогічного персоналу у загальній кількості працівників, %**","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**",Кількість учнів разом,Кількість класів разом,"Фактична наповнюваність класів, учнів на клас**","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)","Видатки на 1 учня, 2018, тис. грн.**","Видатки на 1 клас, тис. грн.** 2018 рік (каса)",Тип школи: міська / сільська,"Разом кількість учнів, та дітей, які здобувають дошкільну освіту","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)","Питома вага непедагогічного персоналу у загальній кількості працівників, %","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %","Фактична наповнюваність класів, учнів на клас","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя"
0,6,3,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ліцей,Луганська область,Троїцький район,с.Демино-Олександрівка,"Відділ освіти, молоді та спорту Троїцької сели...",12.857143,I-III,-1.0,15.0,6.0,2.0,-1.0,-1.0,62,8,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,село,-1.0,-1.0,-1,-1.0,-1.0,0.285714,0.133333,7.750000,4.133333
1,2,2,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ліцей,Луганська область,Троїцький район,с.Лантратівка,"Відділ освіти, молоді та спорту Троїцької сели...",19.666667,I-III,-1.0,17.0,9.0,4.0,-1.0,-1.0,96,11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,село,-1.0,-1.0,-1,-1.0,-1.0,0.346154,0.235294,8.727273,5.647059
2,23,25,0,19,0,0,0,0,0,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,навчально-виховний комплекс,м.Київ,м.Київ. Шевченківський район міста,Шевченківський район міста,Управління освіти Шевченківської районної в мі...,40.625000,I-III,-1.0,53.0,34.0,14.0,-1.0,-1.0,782,27,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,місто,-1.0,-1.0,-1,-1.0,-1.0,0.390805,0.264151,28.962963,14.754717
3,11,10,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,ліцей,Луганська область,Троїцький район,смт Троїцьке,"Відділ освіти, молоді та спорту Троїцької сели...",14.642857,I-III,-1.0,41.0,25.0,5.0,-1.0,-1.0,362,21,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,місто,-1.0,-1.0,-1,-1.0,-1.0,0.378788,0.121951,17.238095,8.829268
4,5,4,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,середня загальноосвітня школа,Сумська область,Глухівський район,смт Шалигине,"Відділ освіти, молоді та спорту Шалигинської с...",9.250000,I-III,-1.0,29.0,22.0,4.0,-1.0,-1.0,149,15,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,місто,-1.0,-1.0,-1,-1.0,-1.0,0.431373,0.137931,9.933333,5.137931


In [25]:
train_df,test_df = train_test_split(X,test_size=0.2)

In [27]:
onehotencode_col = ["eotypename_min", "Ступінь закладу ( I, I-II, I-III)"]

In [28]:
preprocessor = Preprocessing(
    drop_strange_st_data = False,
    column_for_onehot = onehotencode_col
)

In [29]:
preprocess_train_df = preprocessor.process(
    df = train_df,
    data_type = 'train'
)
preprocess_train_df

,sexM,sexF,phys_mat,foreign_phyl,universal,ukr_phyl,technology,bio_phys,economy,history,sport,math,law,other,martial_sport,philosophy,ecology,arts,geography,physics,chem_tech,biology,phys_chem,young_spec,qual_work,hung_lang_max,ukr_lang_max,rus_lang_max,roman_lang_max,pol_lang_max,mold_lang_max,other_lang_max,eoregname_min,eoareaname_min,eotername_min,eoparent_min,mathball_mean,Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,Кількість штатних працівників педагогічного персоналу пенсійного віку,"Питома вага непедагогічного персоналу у загальній кількості працівників, %**","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**",Кількість учнів разом,Кількість класів разом,"Фактична наповнюваність класів, учнів на клас**","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2018рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (план)","Видатки на 1 учня, 2018, тис. грн.**","Видатки на 1 клас, тис. грн.** 2018 рік (каса)",Тип школи: міська / сільська,"Разом кількість учнів, та дітей, які здобувають дошкільну освіту","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)","Питома вага непедагогічного персоналу у загальній кількості працівників, %","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %","Фактична наповнюваність класів, учнів на клас","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя",eotypename_min_гімназія,eotypename_min_колегіум,eotypename_min_ліцей,eotypename_min_навчально-виховне об'єднання,eotypename_min_навчально-виховний комплекс,eotypename_min_середня загальноосвітня школа,eotypename_min_спеціальна загальноосвітня школа,eotypename_min_спеціалізована школа,"Ступінь закладу ( I, I-II, I-III)_I-III","Ступінь закладу ( I, I-II, I-III)_II-III","Ступінь закладу ( I, I-II, I-III)_III"
4435,4,8,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,365,1409,221,16.400000,-1.0,25.0,18.0,3.0,-1.0,-1.0,302,15,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,-1.0,-1.0,-1,-1.0,-1.0,0.418605,0.120000,20.133333,12.080000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3382,10,23,0,0,0,15,0,0,0,0,0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,12,403,1064,574,22.550000,-1.0,52.0,29.0,6.0,-1.0,-1.0,725,31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,-1.0,-1.0,-1,-1.0,-1.0,0.358025,0.115385,23.387097,13.942308,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3394,5,5,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,406,2281,855,34.000000,-1.0,27.0,21.0,3.0,-1.0,-1.0,259,17,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,-1.0,-1.0,-1,-1.0,-1.0,0.437500,0.111111,15.235294,9.592593,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1786,9,3,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,9,492,1158,27,11.285714,-1.0,19.0,27.0,1.0,-1.0,-1.0,335,18,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,-1.0,-1.0,-1,-1.0,-1.0,0.586957,0.052632,18.611111,17.631579,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2943,7,7,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,11,288,1767,552,17.000000,-1.0,22.0,14.0,6.0,-1.0,-1.0,185,11,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,-1.0,-1.0,-1,-1.0,-1.0,0.388889,0.272727,16.818182,8.409091,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3566,12,7,0,0,0,0,0,0,0,19,0,0,0,

In [30]:
preprocess_test_df = preprocessor.process(
    df = test_df,
    data_type = 'test'
)

In [49]:
rf_reg = RandomForestRegressor(
    random_state=42,
    max_depth=10,
    n_estimators=500
)

In [50]:
y_train = preprocess_train_df['mathball100_mean']
X_train = preprocess_train_df.drop(columns=['mathball100_mean'])
rf_reg.fit(X_train,y_train)

RandomForestRegressor(max_depth=10, n_estimators=500, random_state=42)

In [51]:
rf_reg.score(X_train,y_train)

0.6177398763789164

In [52]:
y_test = preprocess_test_df['mathball100_mean']
X_test = preprocess_test_df.drop(columns=['mathball100_mean'])
rf_reg.score(X_test,y_test)

0.36235277190067605

In [53]:
from sklearn.metrics import mean_squared_error
y_pred = rf_reg.predict(X_test)
mean_squared_error(y_test, y_pred)

51.09120759304609

In [54]:
y_pred[:5],y_test[:5]

(array([18.60163036, 19.72134338, 20.17616093, 19.19389334, 15.16080638]),
 4662    19.0
 1207    10.5
 3955    11.2
 3994    23.5
 1458    30.0
 Name: mathball_mean, dtype: float64)

# Classification

In [77]:
X = df[col]
X = X.fillna(-1)

In [78]:
def encode_statuc_binary(x):
    if x > 15:
        return 1
    return 0

X['mathball_status'] = X.mathball100_mean.map(encode_statuc_binary)

In [79]:
train_df,test_df = train_test_split(X, test_size=0.2)

In [80]:
onehotencode_col = ["eotypename_min", "Ступінь закладу ( I, I-II, I-III)"]
preprocessor = Preprocessing(
    drop_strange_st_data = False,
    column_for_onehot = onehotencode_col
)

In [81]:
preprocess_train_df = preprocessor.process(
    df = train_df,
    data_type = 'train'
)
preprocess_test_df = preprocessor.process(
    df = test_df,
    data_type = 'test'
)

In [82]:
rf_cf = RandomForestClassifier(
    random_state=42,
    max_depth=20,
    n_estimators=500
)

In [84]:
y_train = preprocess_train_df['mathball_status']
X_train = preprocess_train_df.drop(columns=['mathball_status', 'mathball100_mean'])
rf_cf.fit(X_train,y_train)

RandomForestClassifier(max_depth=20, n_estimators=500, random_state=42)

In [85]:
rf_cf.score(X_train,y_train)

0.9997839239412273

In [86]:
y_test = preprocess_test_df['mathball_status']
X_test = preprocess_test_df.drop(columns=['mathball_status', 'mathball100_mean'])
rf_cf.score(X_test,y_test)

0.7366148531951641

In [87]:
pred_train_y = rf_cf.predict(X_train)
pred_test_y = rf_cf.predict(X_test)

In [88]:
print('TRAIN DATA:')
print(classification_report(y_train,pred_train_y))
print('\n\n')
print('TEST DATA:')
print(classification_report(y_test,pred_test_y))

TRAIN DATA:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1239
           1       1.00      1.00      1.00      3389

    accuracy                           1.00      4628
   macro avg       1.00      1.00      1.00      4628
weighted avg       1.00      1.00      1.00      4628




TEST DATA:
              precision    recall  f1-score   support

           0       0.59      0.35      0.44       340
           1       0.77      0.90      0.83       818

    accuracy                           0.74      1158
   macro avg       0.68      0.62      0.63      1158
weighted avg       0.71      0.74      0.71      1158

